In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras.utils import np_utils
import math
import numpy as np

Using TensorFlow backend.


In [16]:
teams = pd.read_csv('data/MDataFiles_Stage1/MTeams.csv')
teams = teams[teams['LastD1Season']>2019]
teams = teams[['TeamName','TeamID','FirstD1Season','LastD1Season']]
teams.head(3)

,TeamName,TeamID,FirstD1Season,LastD1Season
0,Abilene Chr,1101,2014,2020
1,Air Force,1102,1985,2020
2,Akron,1103,1985,2020


In [17]:
seasons = pd.read_csv('data/MDataFiles_Stage1/MSeasons.csv').drop(['DayZero'],axis=1)
seasons.head(5)

,Season,RegionW,RegionX,RegionY,RegionZ
0,1985,East,West,Midwest,Southeast
1,1986,East,Midwest,Southeast,West
2,1987,East,Southeast,Midwest,West
3,1988,East,Midwest,Southeast,West
4,1989,East,West,Midwest,Southeast


In [18]:
seeds = pd.read_csv('data/MDataFiles_Stage1/MNCAATourneySeeds.csv')
seeds.head(4)

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260


In [24]:
regions = []
for seas in seeds.iterrows():
    year = seas[1]['Season']
    correctregion = seas[1]['Seed'][0:1]
    column = 'Region' + correctregion
    correctseason = seasons[seasons['Season'] == year]
    actualregion = str(correctseason[column].iloc[0])
    regions.append(actualregion)
seeds['Region'] = regions
seeds.head(4)

,Season,Seed,TeamID,Region,TeamName
0,1985,W01,1207,East,Georgetown
1,1985,W02,1210,East,Georgia Tech
2,1985,W03,1228,East,Illinois
3,1985,W04,1260,East,Loyola-Chicago


In [25]:
names = []
for seas in seeds.iterrows():
    idnum = seas[1]['TeamID']
    correctteam = teams[teams['TeamID'] == idnum]
    teamname = correctteam['TeamName']
    names.append(teamname.iloc[0])
seeds['TeamName'] = names
seeds.head(4)

,Season,Seed,TeamID,Region,TeamName
0,1985,W01,1207,East,Georgetown
1,1985,W02,1210,East,Georgia Tech
2,1985,W03,1228,East,Illinois
3,1985,W04,1260,East,Loyola-Chicago


In [26]:
seeds = seeds[['TeamName','Season', 'TeamID', 'Seed']]
seeds.head()

,TeamName,Season,TeamID,Seed
0,Georgetown,1985,1207,W01
1,Georgia Tech,1985,1210,W02
2,Illinois,1985,1228,W03
3,Loyola-Chicago,1985,1260,W04
4,SMU,1985,1374,W05


In [27]:
cresults = pd.read_csv('data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
cresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [23]:
tresults = pd.read_csv('data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [10]:
def makeloserfirst(df):
    output = pd.DataFrame(columns = ['Season','TeamID1','1Score','TeamID2','2Score'])
    for index, row in df.iterrows():
        if row['WTeamID'] > row['LTeamID']:
            row = list(row[['Season','LTeamID','LScore','WTeamID','WScore']])
        else:
            row = list(row[['Season','WTeamID','WScore','LTeamID','LScore']])
        output.loc[len(output)] = row
    return output
ordered = makeloserfirst(tresults)

In [11]:
ordered.head(2)

,Season,TeamID1,1Score,TeamID2,2Score
0,1985,1116,63,1234,54
1,1985,1120,59,1345,58


In [12]:
stats = pd.read_csv('data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv').drop(['DayNum','NumOT','WLoc'],axis=1)
stats.head()

,Season,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,1104,68,1328,62,27,58,3,14,11,...,10,16,22,10,22,8,18,9,2,20
1,2003,1272,70,1393,63,26,62,8,20,10,...,24,9,20,20,25,7,12,8,6,16
2,2003,1266,73,1437,61,24,58,8,18,17,...,26,14,23,31,22,9,12,2,5,23
3,2003,1296,56,1457,50,18,38,3,9,17,...,22,8,15,17,20,9,19,4,3,23
4,2003,1400,77,1208,71,30,61,6,14,11,...,16,17,27,21,15,12,10,7,1,14


In [13]:
cols = ['Season', 'TeamID', 'TScore', 'AScore',
               'TFGM', 'TFGA', 'TFGM3', 'TFGA3', 'TFTM', 'TFTA', 'TOR', 'TDR', 'TAst', 
                'TTO', 'TStl', 'TBlk', 'TPF','AFGM', 'AFGA', 'AFGM3', 'AFGA3', 'AFTM',
                'AFTA', 'AOR', 'ADR', 'AAst', 'ATO', 'AStl', 'ABlk', 'APF']
teamaverages = pd.DataFrame(columns = cols)
for year in stats['Season'].unique():   
    print(year)
    for team in teams['TeamID']:
        wtemp = stats[(stats['WTeamID'] == team) & (stats['Season'] == year)]
        ltemp = stats[(stats['LTeamID'] == team) & (stats['Season'] == year)]
        temp = pd.concat([wtemp, ltemp])
    
        allgames = pd.DataFrame(columns = cols)
        for index,row in temp.iterrows():
            if row['LTeamID'] == team: #if the team lost this game
                row = row[['Season', 'LTeamID', 'LScore', 'WScore',
               'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 
                'LTO', 'LStl', 'LBlk', 'LPF','WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM',
                'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
            elif row['WTeamID'] == team:
                row = row[['Season', 'WTeamID', 'WScore', 'LScore',
               'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst',
               'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM',
               'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
            
            row.index = cols
            row = row.values.tolist()
            allgames.loc[len(allgames)] = row
        
        averages = []
        for co in allgames:
            averages.append(round(allgames[co].mean(),1))
        
        teamaverages.loc[len(teamaverages)] = averages

2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [14]:
teamaverages = teamaverages.dropna().reset_index(drop=True)
teamaverages.head()

,Season,TeamID,TScore,AScore,TFGM,TFGA,TFGM3,TFGA3,TFTM,TFTA,...,AFGA3,AFTM,AFTA,AOR,ADR,AAst,ATO,AStl,ABlk,APF
0,2003.0,1102.0,57.2,57.0,19.1,39.8,7.8,20.8,11.1,17.1,...,12.4,13.7,19.2,9.6,20.1,9.1,13.0,5.4,1.6,18.4
1,2003.0,1103.0,78.8,78.1,27.1,55.9,5.4,16.1,19.0,25.9,...,18.4,15.9,22.1,12.0,22.0,15.5,15.3,6.4,2.9,22.4
2,2003.0,1104.0,69.3,65.0,24.0,57.2,6.4,19.9,14.9,20.9,...,19.1,12.1,17.1,10.9,22.6,11.7,13.9,5.5,3.2,19.2
3,2003.0,1105.0,71.8,76.7,24.4,61.6,7.6,20.8,15.4,21.8,...,17.5,16.4,24.5,13.2,26.4,15.8,18.8,9.4,4.2,19.1
4,2003.0,1106.0,63.6,63.8,23.4,55.3,6.1,17.6,10.6,16.5,...,15.2,15.5,22.0,11.3,22.4,11.8,15.1,8.8,3.2,16.1


In [15]:
teamaverages.to_csv('teamaverages.csv',index=False)